In [1]:
pip install tf-keras


^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install transformers torch datasets scikit-learn pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from torch.nn.functional import softmax

In [4]:
# 🔹 Load pre-trained tokenizer and model
model_name = "distilbert-base-uncased"  # You can also use "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def load_data(filepath):
    df = pd.read_csv(filepath)
    df = df[['text', 'label']]  # Keep only relevant columns
    return df

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [7]:
def prepare_dataset(filepath):
    df = load_data(filepath)
    train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)
    
    train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()})
    test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()})

    return train_dataset.map(tokenize_function, batched=True), test_dataset.map(tokenize_function, batched=True)

In [8]:
def train_transformer_model(dataset_path):
    train_dataset, test_dataset = prepare_dataset(dataset_path)
    
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset
    )

    trainer.train()

In [9]:
dataset_path = r"C:\Users\msi\Desktop\NLP\Dataset\Generated_Fakenews_dataset.csv"  # Update with your dataset path
train_transformer_model(dataset_path)

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

C:\Python312\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [12]:
import torch
from sklearn.metrics import classification_report
from torch.nn.functional import softmax
from tqdm import tqdm  # Progress bar for evaluation


import pandas as pd

# Load dataset (update path if necessary)
df_test = pd.read_csv(dataset_path)  # Ensure this file exists
df_test = df_test[["text", "label"]]  # Keep only relevant columns


# Function to make predictions
def predict_news_transformer(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
    tokens = tokenizer(text, truncation=True, return_tensors="pt").to(device)  
    model.to(device)  
    model.eval()  # Ensure model is in evaluation mode

    with torch.no_grad():
        output = model(**tokens)  

    probabilities = softmax(output.logits, dim=1)[0].tolist()
    prediction = torch.argmax(output.logits, dim=1).item()
    
    return 1 if prediction == 1 else 0  # 1 for REAL, 0 for FAKE

# Load your test dataset (Assuming you have a DataFrame 'df_test' with 'text' and 'label' columns)
true_labels = df_test["label"].tolist()  # Ground truth labels
predicted_labels = []

# Perform batch prediction
for text in tqdm(df_test["text"], desc="Evaluating Model"):
    predicted_labels.append(predict_news_transformer(text))

# Generate classification report
print(classification_report(true_labels, predicted_labels, target_names=["FAKE", "REAL"]))


Evaluating Model: 100%|███████████████████████████████████████████████████████████| 9000/9000 [00:55<00:00, 162.61it/s]

              precision    recall  f1-score   support

        FAKE       1.00      1.00      1.00      4500
        REAL       1.00      1.00      1.00      4500

    accuracy                           1.00      9000
   macro avg       1.00      1.00      1.00      9000
weighted avg       1.00      1.00      1.00      9000



In [13]:
import torch
from torch.nn.functional import softmax
sample_text = "Apples grow on trees"
result = predict_news_transformer(sample_text)
print(f"Prediction: {result['prediction']}")
print(f"Confidence: {result['confidence']:.2%}")

TypeError: 'int' object is not subscriptable